In [ ]:
import os
import time
from openai import OpenAI
from requests_oauthlib import OAuth1Session

# Fetch the API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
twitter_consumer_key = os.getenv('TWITTER_CONSUMER_KEY1')
twitter_consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET1')
twitter_access_token = os.getenv('TWITTER_ACCESS_TOKEN1')
twitter_access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET1')

if not openai_api_key:
    raise ValueError("No OpenAI API key found. Set the OPENAI_API_KEY environment variable.")
if not twitter_consumer_key or not twitter_consumer_secret or not twitter_access_token or not twitter_access_token_secret:
    raise ValueError("Twitter API keys not found. Set the environment variables for Twitter.")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

# Function to generate response content for a reply
def generate_response_to_reply(reply_content):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a friendly and informative Bitcoin enthusiast responding to a tweet."},
            {"role": "user", "content": f"Reply to this tweet in an informative and engaging way: {reply_content}"}
        ]
    )
    return response.choices[0].message.content

# Function to fetch recent mentions/replies
def fetch_mentions(consumer_key, consumer_secret, access_token, access_token_secret):
    url = 'https://api.twitter.com/2/users/:id/mentions'
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )
    params = {
        'tweet.fields': 'conversation_id,created_at,author_id,in_reply_to_user_id',
        'max_results': 5
    }
    response = oauth.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")
    return response.json()

# Function to post reply tweets
def post_reply(consumer_key, consumer_secret, access_token, access_token_secret, reply_content, in_reply_to_tweet_id):
    url = 'https://api.twitter.com/2/tweets'
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )
    payload = {
        'text': reply_content,
        'in_reply_to_tweet_id': in_reply_to_tweet_id
    }
    response = oauth.post(url, json=payload)
    if response.status_code != 201:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")
    return response.json()

# Function to continuously check for replies and respond
def reply_to_mentions():
    last_checked = time.time()
    while True:
        try:
            mentions = fetch_mentions(twitter_consumer_key, twitter_consumer_secret, twitter_access_token, twitter_access_token_secret)
            for mention in mentions.get('data', []):
                reply_content = mention.get('text')
                tweet_id = mention.get('id')
                # Generate a response to the mention
                response_content = generate_response_to_reply(reply_content)
                # Post the generated response
                post_reply(twitter_consumer_key, twitter_consumer_secret, twitter_access_token, twitter_access_token_secret, response_content, tweet_id)
                print(f"Replied: {response_content}")
            time.sleep(300)  # Check for new mentions every 5 minutes
        except Exception as e:
            print(f"Error fetching or responding to mentions: {e}")
            time.sleep(300)  # Wait 5 minutes before retrying

# Start replying to mentions
reply_to_mentions()


Error fetching or responding to mentions: Request returned an error: 403 {"client_id":"29172109","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API Access","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}
Error fetching or responding to mentions: Request returned an error: 403 {"client_id":"29172109","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"

In [ ]:
import os
import time
import requests
from openai import OpenAI

# Fetch the API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
twitter_bearer_token = ('')

if not openai_api_key:
    raise ValueError("No OpenAI API key found. Set the OPENAI_API_KEY environment variable.")
if not twitter_bearer_token:
    raise ValueError("Twitter Bearer Token not found. Set the TWITTER_BEARER_TOKEN environment variable.")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

# Function to generate response content for a reply
def generate_response_to_reply(reply_content):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a friendly and informative Bitcoin enthusiast responding to a tweet."},
            {"role": "user", "content": f"Reply to this tweet in an informative and engaging way: {reply_content}"}
        ]
    )
    return response.choices[0].message.content

# Function to fetch recent mentions/replies using Bearer Token
def fetch_mentions(bearer_token):
    url = 'https://api.twitter.com/2/users/:id/mentions'
    headers = {
        'Authorization': f'Bearer {bearer_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'tweet.fields': 'conversation_id,created_at,author_id,in_reply_to_user_id',
        'max_results': 5
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")
    return response.json()

# Function to post reply tweets using Bearer Token
def post_reply(bearer_token, reply_content, in_reply_to_tweet_id):
    url = 'https://api.twitter.com/2/tweets'
    headers = {
        'Authorization': f'Bearer {bearer_token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'text': reply_content,
        'reply': {'in_reply_to_tweet_id': in_reply_to_tweet_id}
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 201:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")
    return response.json()

# Function to continuously check for replies and respond
def reply_to_mentions():
    while True:
        try:
            mentions = fetch_mentions(twitter_bearer_token)
            for mention in mentions.get('data', []):
                reply_content = mention.get('text')
                tweet_id = mention.get('id')
                # Generate a response to the mention
                response_content = generate_response_to_reply(reply_content)
                # Post the generated response
                post_reply(twitter_bearer_token, response_content, tweet_id)
                print(f"Replied: {response_content}")
            time.sleep(300)  # Check for new mentions every 5 minutes
        except Exception as e:
            print(f"Error fetching or responding to mentions: {e}")
            time.sleep(300)  # Wait 5 minutes before retrying

# Start replying to mentions
reply_to_mentions()


Error fetching or responding to mentions: Request returned an error: 403 {"client_id":"29172109","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API Access","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}
Error fetching or responding to mentions: Request returned an error: 403 {"client_id":"29172109","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"